In [ ]:
# -*- coding: utf-8 -*-
from pandas import DataFrame,Series
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import copy
import Ipynb_importer
from utils import str2stamp

nameList = [ '机构设立（变更）登记信息.csv',
             '企业税务登记信息.csv',
             '企业非正常户认定.csv',
             '双公示-法人行政许可信息.csv',
             '双打办打击侵权假冒处罚案件信息.csv',
             '法人行政许可注（撤、吊）销信息.csv',
             '许可资质年检信息.csv',
             '资质登记（变更）信息.csv',
             '招聘数据.csv',
             '分支机构信息.csv',
             '企业表彰荣誉信息.csv',
            '企业基本信息&高管信息&投资信息.csv',
            # '失信被执行人名单.csv',
            #'双公示-法人行政处罚信息.csv',
           ]

featureList = [['企业名称', '信息提供部门编码','统一社会信用代码','种类', '法定代表人姓名','注册（开办）资金', '实收资金', '企业类型代码' , '所属行业代码', '发证机关名称','行政区划' , '是否有经纬度'],
               ['企业名称',  '信息提供部门编码','税务管理码', '纳税人识别号', '法定代表人证件名称', '登记注册类型',  '审核结果',   '审核单位'],
               ['企业名称', '信息提供部门编码',  '税务管理码', '纳税人识别号',  '纳税人状态',  '登记注册类型','管理机构'],
               ['企业名称', '数据来源',  '信息提供部门编码',  '行政许可编码',  '审批类别',  '法定代表人名称',  '行政相对人名称', '地方编码'],
               ['企业名称',  '信息提供部门编码',  '被处罚的自然人身份证号',  '被处罚的企业法定代表人身份证号'],
               ['企业名称' , '种类',  '许可证编号',  '许可证名称',  '注（撤、吊）销原因',  '注（撤、吊）销批准机关全称',  '权力编码',  '信息提供部门编码' ],
               ['企业名称'  ,'信息提供部门编码',  '年检年度', '年检结果',  '年检事项名称'],
               ['企业名称' ,'信息提供部门编码',   '种类',   '资质名称',  '认定机关全称']
              ]
class Preprocess():
    
    def __init__(self,input_path,nameList):
        self.input_path=input_path
        self.nameList=nameList

        pass

    def Basedata(self):
        '''
        处理训练集和测试集

        '''
        print('\n....Basedata() 处理基础数据......\n')
        #'机构设立（变更）登记信息.csv'
        f1=pd.read_csv(open(self.input_path+nameList[0],'r', encoding='UTF-8'))
        f1['（变更）核准日期']=str2stamp(f1['（变更）核准日期'])
        f1=f1.sort_values(by=['（变更）核准日期'])
        f1.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f1.to_csv(self.input_path+'retrain/'+'%s'%nameList[0],encoding='gbk',index=False)

        # '企业税务登记信息.csv',
        f2=pd.read_csv(open(self.input_path+nameList[1],'r', encoding='UTF-8'))
        f2=f2[f2['审核结果']!='江苏省苏州地方税务局']
        f2.to_csv(self.input_path+'retrain/'+'%s'%nameList[1],encoding='gbk',index=False)

        # '企业非正常户认定.csv',
        f3=pd.read_csv(open(self.input_path+nameList[2],'r', encoding='UTF-8'))
        f3.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f3.to_csv(self.input_path+'retrain/'+'%s'%nameList[2],encoding='UTF-8',index=False)


        # '双公示-法人行政许可信息.csv',
        f4=pd.read_csv(open(self.input_path+nameList[3],'r', encoding='UTF-8'))
        f4.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f4.to_csv(self.input_path+'retrain/'+'%s'%nameList[3],encoding='gbk',index=False)

        #'双打办打击侵权假冒处罚案件信息.csv',
        f5=pd.read_csv(open(self.input_path+nameList[4],'r', encoding='UTF-8'))
        f5.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f5.to_csv(self.input_path+'retrain/'+'%s'%nameList[4],encoding='UTF-8',index=False)


        #'法人行政许可注（撤、吊）销信息.csv',
        f6=pd.read_csv(open(self.input_path+nameList[5],'r', encoding='UTF-8'))
        f6.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f6.to_csv(self.input_path+'retrain/'+'%s'%nameList[5],encoding='gbk',index=False)


        #'许可资质年检信息.csv',
        f7=pd.read_csv(open(self.input_path+nameList[6],'r', encoding='UTF-8'))
        f7.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f7.to_csv(self.input_path+'retrain/'+'%s'%nameList[6],encoding='gbk',index=False)


        #'资质登记（变更）信息.csv',
        f8=pd.read_csv(open(self.input_path+nameList[7],'r', encoding='UTF-8'))
        f8.drop_duplicates(subset='企业名称',keep='last',inplace=True)
        f8.to_csv(self.input_path+'retrain/'+'%s'%nameList[7],encoding='gbk',index=False)
        print('\n......Basedata() done........\n')
        pass
    
    
    
    def Middata(self,mid_path,flag):

        '''
        合表，区别重复feature
        flag:R1train,R1test,R2train,R2test
        '''
        print('\n....Middata() 合表......\n')
        base=pd.read_csv(open(self.input_path+'retrain/'+nameList[0],'r', encoding='gbk'))
        database=base[featureList[0]]

        for i in range(1,8):
            try:
                data=pd.read_csv(self.input_path+'retrain/'+nameList[i],encoding='gbk',engine='python')
            except Exception as err: 
                data=pd.read_csv(self.input_path+'retrain/'+nameList[i],encoding='utf-8',engine='python')
            print('file%s done'%nameList[i])
            database=pd.merge(database,data[featureList[i]],on='企业名称',how='left')

        #查询第一列
        array1=database['信息提供部门编码_x'].iloc[:,[0]].rename(columns={'信息提供部门编码_x':'dp1'})
        array2=database['信息提供部门编码_x'].iloc[:,[1]].rename(columns={'信息提供部门编码_x':'dp2'})
        array3=database['信息提供部门编码_x'].iloc[:,[2]].rename(columns={'信息提供部门编码_x':'dp3'})
        array4=database['信息提供部门编码_x'].iloc[:,[3]].rename(columns={'信息提供部门编码_x':'dp4'})
        array5=database['信息提供部门编码_y'].iloc[:,[0]].rename(columns={'信息提供部门编码_y':'dp5'})
        array6=database['信息提供部门编码_y'].iloc[:,[1]].rename(columns={'信息提供部门编码_y':'dp6'})
        array7=database['信息提供部门编码_y'].iloc[:,[2]].rename(columns={'信息提供部门编码_y':'dp7'})
        array8=database['信息提供部门编码_y'].iloc[:,[3]].rename(columns={'信息提供部门编码_y':'dp8'})


        test1=database.drop(['信息提供部门编码_x','信息提供部门编码_y'],axis=1)

        new_base=pd.concat([test1,array1,array2,array3,array4,array5,array6,array7,array8],axis=1)
        
        #print(new_base.shape)
        df1,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12=self.Addfeature()
        df_list=[df1,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
        #print(sum([x.shape[1] for x in df_list]))
        
        for item in df_list:
            new_base=pd.merge(new_base,item,on=['企业名称'],how='left')
            #print(new_base.shape[1])

        if flag=='R1train' or flag=='R2train':
            l1=pd.read_csv(open(self.input_path+'失信被执行人名单.csv','r', encoding='UTF-8'))
            l1['L1']=1
            l2=pd.read_csv(open(self.input_path+'双公示-法人行政处罚信息.csv','r', encoding='UTF-8'))
            l2['L2']=1
            new_base=pd.merge(new_base,l1,on='企业名称',how='left')
            new_base=pd.merge(new_base,l2,on='企业名称',how='left')
            new_base

            new_base['L1'].fillna(0,inplace=True)
            new_base['L2'].fillna(0,inplace=True)
            new_base

            new_base['L1']=new_base['L1'].astype(int)
            new_base['L2']=new_base['L2'].astype(int)
        

        try:
            new_base.to_csv(mid_path+'mid%s.csv'%flag,encoding='gbk',index=False)
        except Exception as err: 
            new_base.to_csv(mid_path+'mid%s.csv'%flag,encoding='utf-8',index=False)

        print('\n....Middata() done......\n')
        pass

    def Addfeature(self):
        '''
        添加统计特征
        '''
        print('\n....Addfeature() 添加统计特征.....\n')
        f1=pd.read_csv(open(self.input_path+nameList[0],'r', encoding='UTF-8'))
        f1['变更次数']=1
        df1=f1[['企业名称','变更次数']].groupby(['企业名称']).sum().reset_index()
        df1
        f3=pd.read_csv(open(self.input_path+nameList[2],'r', encoding='UTF-8'))
        f3['非正常用户次数']=1
        df3=f3[['企业名称','非正常用户次数']].groupby(['企业名称']).sum().reset_index()
        df3

        f4=pd.read_csv(open(self.input_path+nameList[3],'r', encoding='UTF-8'))
        f4['许可项数']=1
        df4=f4[['企业名称','许可项数']].groupby(['企业名称']).sum().reset_index()
        df4['是否有许可']=1
        df4


        f5=pd.read_csv(open(self.input_path+nameList[4],'r', encoding='UTF-8'))
        f5['被罚次数']=1
        df5=f5[['企业名称','被罚次数']].groupby(['企业名称']).sum().reset_index()
        df5['是否被罚']=1
        df5

        f6=pd.read_csv(open(self.input_path+nameList[5],'r', encoding='UTF-8'))
        f6['注销次数']=1
        df6=f6[['企业名称','注销次数']].groupby(['企业名称']).sum().reset_index()
        df6['是否注销']=1
        df6

        f7=pd.read_csv(open(self.input_path+nameList[6],'r', encoding='UTF-8'))
        f7['年检次数']=1
        df7=f7[['企业名称','年检次数']].groupby(['企业名称']).sum().reset_index()
        df7['是否年检']=1
        df7


        f8=pd.read_csv(open(self.input_path+nameList[7],'r', encoding='UTF-8'))
        f8['资质项数']=1
        df8=f8[['企业名称','资质项数']].groupby(['企业名称']).sum().reset_index()
        df8['有无资质']=1
        df8

        f9=pd.read_csv(open(self.input_path+nameList[8],'r', encoding='UTF-8'))
        f9['招聘条数']=1
        df91=f9[['企业名称','招聘条数']].groupby(['企业名称']).sum().reset_index()

        df92=f9[['企业名称','网站名称']].drop_duplicates(['企业名称','网站名称'])
        df92['发布渠道数']=1
        df92=df92[['企业名称','发布渠道数']].groupby(['企业名称']).sum().reset_index()

        df93=f9[['企业名称','工作地点']].drop_duplicates(['企业名称','工作地点'])
        df93['工作地点数']=1
        df93=df93[['企业名称','工作地点数']].groupby(['企业名称']).sum().reset_index()
        df93


        f9['招聘人数_合计']=f9['招聘人数'].astype(str)
        f9['招聘人数_合计']=f9['招聘人数_合计'].apply(lambda x: x if x[-1]!='人' else x[0:-1])
        f9['招聘人数_合计']=f9['招聘人数_合计'].apply(lambda x: '0' if x=='nan' else x)
        f9['招聘人数_合计']=f9['招聘人数_合计'].apply(lambda x: '5' if x=='若干' else x).astype(int)
        df94=f9[['企业名称','招聘人数_合计']].groupby(['企业名称']).sum().reset_index()
        df94


        df9=pd.merge(df91,df92,on=['企业名称'],how='left')
        df9=pd.merge(df9,df93,on=['企业名称'],how='left')
        df9=pd.merge(df9,df94,on=['企业名称'],how='left')
        df9['有无招聘信息']=1


        f10=pd.read_csv(open(self.input_path+nameList[9],'r', encoding='UTF-8'))
        f10['分支机构数']=1
        df101=f10[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index()
        df101

        df102=f10.dropna(subset=['分支死亡时间'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'死亡分支数'})
        df102

        df103=f10[['企业名称','分支机构省份','分支机构数']].drop_duplicates(['企业名称','分支机构省份'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'分支省份数'})
        df103

        df104=f10[['企业名称','分支行业门类','分支机构数']].drop_duplicates(['企业名称','分支行业门类'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'分支行业门类数'})
        df104


        df105=f10[['企业名称','分支行业代码','分支机构数']].drop_duplicates(['企业名称','分支行业代码'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'分支行业代码数'})
        df105

        df106=f10[['企业名称','分支机构区县','分支机构数']].drop_duplicates(['企业名称','分支机构区县'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'分支机构区县数'})
        df106

        df107=f10[['企业名称','分支机构类型','分支机构数']].drop_duplicates(['企业名称','分支机构类型'])[['企业名称','分支机构数']].groupby(['企业名称']).sum().reset_index().rename(columns={'分支机构数':'分支机构类型数'})

        df107

        df10=pd.merge(df101,df102,on=['企业名称'],how='left')
        df10=pd.merge(df10,df103,on=['企业名称'],how='left')
        df10=pd.merge(df10,df104,on=['企业名称'],how='left')
        df10=pd.merge(df10,df105,on=['企业名称'],how='left')
        df10=pd.merge(df10,df106,on=['企业名称'],how='left')
        df10=pd.merge(df10,df107,on=['企业名称'],how='left')
        df10.fillna(0,inplace=True)
        df10



        f11=pd.read_csv(open(self.input_path+nameList[10],'r', encoding='UTF-8'))
        f11['表彰次数']=1
        df11=f11[['企业名称','表彰次数']].groupby(['企业名称']).sum().reset_index()
        df11['是否表彰']=1
        df11

        f12=pd.read_csv(open(self.input_path+nameList[11],'r', encoding='UTF-8'))
        f12['人数']=1
        df121=f12[['企业名称','姓名','人数']].drop_duplicates(['企业名称','姓名'])[['企业名称','人数']].groupby(['企业名称']).sum().reset_index().rename(columns={'人数':'高管人数'})
        df122=f12[['企业名称','投资人','人数']].drop_duplicates(['企业名称','投资人'])[['企业名称','人数']].groupby(['企业名称']).sum().reset_index().rename(columns={'人数':'投资人数'})

        df123=f12[['企业名称','投资人','出资比例']].drop_duplicates(['企业名称','投资人']).fillna(0)[['企业名称','出资比例']].groupby(['企业名称']).sum().reset_index()

        df12=pd.merge(df121,df122,on=['企业名称'],how='left')
        df12=pd.merge(df12,df123,on=['企业名称'],how='left')
        df12
        print('\n....Addfeature() done......\n')
        
        return df1,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12
